In [1]:
import pandas as pd
import spacy
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import RecognizerResult
from presidio_analyzer import AnalyzerEngine, Pattern, PatternRecognizer, RecognizerRegistry
from presidio_analyzer.nlp_engine import NlpEngineProvider, NlpArtifacts
from typing import List
import multiprocessing
from tqdm import tqdm
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [8]:
data = pd.read_csv("../data/clean_data.csv", index_col=0)

In [3]:
data.dtypes

fecha_de_ingreso             object
dependencia                  object
organo_de_gobierno           object
descripcion_solicitud        object
estatus                      object
medio_entrada                object
otros_datos                  object
fecha_limite_de_respuesta    object
respuesta                    object
texto_de_la_respuesta        object
pais                         object
estado                       object
fecha_respuesta              object
dtype: object

In [15]:
data['fecha_de_ingreso'] = pd.to_datetime(data['fecha_de_ingreso'])
data['fecha_limite_de_respuesta'] = pd.to_datetime(data['fecha_limite_de_respuesta'])
data['fecha_respuesta'] = pd.to_datetime(data['fecha_respuesta'])
data['descripcion_solicitud'] = data['descripcion_solicitud'].astype('string')
data["otros_datos"] = data["otros_datos"].astype("string")
data["dependencia"] = data["dependencia"].astype("string")
data["organo_de_gobierno"] = data["organo_de_gobierno"].astype("string")
data["estatus"] = data["estatus"].astype("string")
data["medio_entrada"] = data["medio_entrada"].astype("string")
data["respuesta"] = data["respuesta"].astype("string")
data["texto_de_la_respuesta"] = data["texto_de_la_respuesta"].astype("string")
data["pais"] = data["pais"].astype("string")
data["estado"] = data["estado"].astype("string")

In [5]:
data.dtypes

fecha_de_ingreso             datetime64[ns]
dependencia                          string
organo_de_gobierno                   string
descripcion_solicitud                string
estatus                              string
medio_entrada                        string
otros_datos                          string
fecha_limite_de_respuesta    datetime64[ns]
respuesta                            string
texto_de_la_respuesta                string
pais                                 string
estado                               string
fecha_respuesta              datetime64[ns]
dtype: object

## Testado

In [2]:
configuration = {
    "nlp_engine_name": "spacy",
    "models": [
        {"lang_code": "es", "model_name": "es_core_news_sm"},
        {"lang_code": "en", "model_name": "en_core_web_lg"},
    ],
}
provider = NlpEngineProvider(nlp_configuration=configuration)
nlp_engine_with_spanish = provider.create_engine()
analyzer = AnalyzerEngine(nlp_engine=nlp_engine_with_spanish, supported_languages=["en", "es"])

In [3]:
# Define el patrón regex en un objeto `Pattern` de Presidio:
curp_pattern = Pattern(name="custom_pattern", regex=r"\b[a-zA-Z0-9]{18}\b", score=0.5)

# Define el reconocedor con uno o más patrones
curp_recognizer = PatternRecognizer(
    supported_entity="CURP", patterns=[curp_pattern]
)
# Prueba el reconocedor en sí mismo
#text = "Este es un ejemplo de texto con una cadena de 18 caracteres: ABC123xyz456789012"

#custom_result = curp_recognizer.analyze(text=text, entities=["CURP"])

#print("Resultado:")
#print(custom_result)

In [4]:
# Define el patrón regex en un objeto `Pattern` de Presidio
rfc_pattern = Pattern(name="custom_pattern", regex=r"\b[A-Z]{4}\d{7}[A-Z]\d\b", score=0.5)

# Define el reconocedor con uno o más patrones
rfc_recognizer = PatternRecognizer(
    supported_entity="RFC", patterns=[rfc_pattern]
)
# Prueba el reconocedor en sí mismo
#text = "Este es un ejemplo de texto con una cadena de 18 caracteres: TRSG9302078A8"

#custom_result = rfc_recognizer.analyze(text=text, entities=["RFC"])

#print("Resultado:")
#print(custom_result)

In [5]:
# Add the recognizer to the existing list of recognizers
analyzer.registry.add_recognizer(curp_recognizer)
analyzer.registry.add_recognizer(rfc_recognizer)

In [6]:
engine = AnonymizerEngine()

In [11]:
#texto = "JISD9308096A5 que tal me gustaria saber la estadistica de delincuencia que existe en la COLONIA ÁLAMOS, JISD930809hdfmnv04 DELEGACION BENITO JUAREZ, ademas de la estadistica de prostitución y drogadicción de la misma colonia que esta dentro de esa delegación. Mi nombre es Eloisa Huerta Lara, mi domicilio es Segovia 59 col. álamos, la informacion la solicito por correo electronico, el cual es magycal_onpu@hotmail.com, eloisahl@yahoo.com.mx, o por este mismo mediO.gracias"
#resultados0 = analyzer.analyze(text=texto, language="es")
#resultados1 = rfc_recognizer.analyze(text=texto, entities=["RFC"])
#resultados2 = curp_recognizer.analyze(text=texto, entities=["CURP"])
#print(resultados2)

In [12]:
#resultados_combinados = resultados0 + resultados1 + resultados2

In [13]:
#testado = engine.anonymize(text=texto, analyzer_results=resultados_combinados)
#print(testado.text)

In [11]:
def anonymize_df_column(df, column):
    """
    This function takes a dataframe and a column name as input, 
    and anonymizes the values in the specified column.
    
    Parameters:
    df (pd.DataFrame): The dataframe containing the column to be anonymized.
    column (str): The name of the column to be anonymized.
    
    Returns:
    df (pd.DataFrame): The dataframe with the anonymized column.
    """
    
    # Define a function for anonymization
    def anonymize_text(text):
        resultados0 = analyzer.analyze(text=text, language="es")
        resultados1 = rfc_recognizer.analyze(text=text, entities=["RFC"])
        resultados2 = curp_recognizer.analyze(text=text, entities=["CURP"])
        resultados_combinados = resultados0 + resultados1 + resultados2
        texto_anonimizado = engine.anonymize(text=text, analyzer_results=resultados_combinados).text
        return texto_anonimizado
    
    # Apply the anonymization function to the specified column
    df[column] = df[column].apply(anonymize_text)
    
    return df

In [32]:
sample_df = data.sample(n=10000, random_state=42)
sample_df.dtypes

fecha_de_ingreso             datetime64[ns]
dependencia                          string
organo_de_gobierno                   string
descripcion_solicitud                string
estatus                              string
medio_entrada                        string
otros_datos                          string
fecha_limite_de_respuesta    datetime64[ns]
respuesta                            string
texto_de_la_respuesta                string
pais                                 string
estado                               string
fecha_respuesta              datetime64[ns]
dtype: object

In [12]:
prueba = anonymize_df_column(sample_df, "descripcion_solicitud")

In [42]:
def anonymize_df_column(df, column):
    """
    This function takes a dataframe and a column name as input, 
    and anonymizes the values in the specified column.
    
    Parameters:
    df (pd.DataFrame): The dataframe containing the column to be anonymized.
    column (str): The name of the column to be anonymized.
    
    Returns:
    df (pd.DataFrame): The dataframe with the anonymized column.
    """
    
    def anonymize_text(text):
        if isinstance(text, str):
            resultados0 = analyzer.analyze(text=text, language="es")
            resultados1 = rfc_recognizer.analyze(text=text, entities=["RFC"])
            resultados2 = curp_recognizer.analyze(text=text, entities=["CURP"])
            resultados_combinados = resultados0 + resultados1 + resultados2
            texto_anonimizado = engine.anonymize(text=text, analyzer_results=resultados_combinados).text
            return texto_anonimizado
        elif isinstance(text, pd.Series):
            return text.apply(anonymize_text)
        else:
            return text

    
    return df


In [43]:
aber = anonymize_df_column(sample_df, "descripcion_solicitud")

In [47]:
aber.to_csv("aber.csv")